# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770731029709                   -0.53    8.0         
  2   -2.772138202134       -2.85       -1.30    1.0    177ms
  3   -2.772170158597       -4.50       -2.66    1.0    151ms
  4   -2.772170721203       -6.25       -4.08    2.0    208ms
  5   -2.772170722966       -8.75       -4.74    2.0    173ms
  6   -2.772170723013      -10.33       -5.74    1.0    160ms
  7   -2.772170723015      -11.71       -6.13    2.0    196ms
  8   -2.772170723015      -13.15       -6.89    1.0    167ms
  9   -2.772170723015   +  -14.75       -7.87    2.0    193ms
 10   -2.772170723015      -13.80       -7.93    2.0    204ms
 11   -2.772170723015   +  -14.07       -9.35    1.0    166ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770772425704                   -0.52    9.0         
  2   -2.7

1.7735579227575207

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770556761967                   -0.53    9.0         
  2   -2.772047373631       -2.83       -1.29    1.0    146ms
  3   -2.772082271361       -4.46       -2.74    1.0    184ms
  4   -2.772083407127       -5.94       -3.57    2.0    168ms
  5   -2.772083416903       -8.01       -4.02    1.0    191ms
  6   -2.772083417735       -9.08       -5.11    1.0    154ms
  7   -2.772083417807      -10.15       -5.21    2.0    193ms
  8   -2.772083417811      -11.42       -6.44    1.0    160ms
  9   -2.772083417811      -13.36       -7.22    2.0    200ms
 10   -2.772083417811   +  -14.88       -7.13    1.0    172ms
 11   -2.772083417811      -13.82       -7.99    2.0    191ms
 12   -2.772083417811   +  -13.75       -8.43    1.0    174ms

Polarizability via ForwardDiff:       1.7725349641176986
Polarizability via finite difference: 1.7735579227575207
